In [ ]:
import os
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

In [ ]:
# Step 1: 读取 Borough 边界数据
# geojson_url = "https://services.arcgis.com/OLiydejKCZTGhvWg/arcgis/rest/services/London_Boroughs/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=geojson"
# boroughs = gpd.read_file(geojson_url)

path = "../data/Borough_Boundaries.geojson"
boroughs = gpd.read_file(path)

boroughs = boroughs[["name", "gss_code", "geometry"]].rename(columns={"name": "borough"})

print(boroughs)
print(boroughs.columns)

In [ ]:
#设置 OSMnx 配置
ox.settings.log_console = False
ox.settings.use_cache = True

In [ ]:
#初始化结果列表
results = []

In [ ]:
#遍历 Borough
for idx, row in tqdm(boroughs.iterrows(), total=len(boroughs), desc="Processing boroughs"):
    borough_name = row["borough"]
    gss_name = row["gss_code"]
    geometry = row["geometry"]

    try:
        print(f"🚦 Processing: {borough_name}")
        
        # 获取 borough 范围内的机动车路网
        G = ox.graph_from_polygon(geometry, network_type="drive", simplify=True)

        # 节点中心性
        betweenness = nx.betweenness_centrality(G, weight="length", k=100, seed=42)
        degree = dict(G.degree())
        nx.set_node_attributes(G, betweenness, "betweenness")
        nx.set_node_attributes(G, degree, "degree")

        # 构造带 u-v 信息的边表
        edge_data = []
        for u, v, key, data in G.edges(keys=True, data=True):
            edge_data.append({
                "u": u,
                "v": v,
                "key": key,
                "geometry": data.get("geometry", None),
                "betweenness": (G.nodes[u]["betweenness"] + G.nodes[v]["betweenness"]) / 2,
                "degree": (G.nodes[u]["degree"] + G.nodes[v]["degree"]) / 2
            })
        edges_df = gpd.GeoDataFrame(edge_data, geometry="geometry", crs="EPSG:4326")

        # 聚合
        summary = {
            "borough": borough_name,
            "gss_code": gss_name,
            "mean_betweenness": edges_df["betweenness"].mean(),
            "max_betweenness": edges_df["betweenness"].max(),
            "mean_degree": edges_df["degree"].mean(),
            "max_degree": edges_df["degree"].max(),
            "edge_count": len(edges_df)
        }
        results.append(summary)

    except Exception as e:
        print(f"Failed for {borough_name}: {e}")
        continue

In [ ]:
df_results = pd.DataFrame(results)
df_results.to_csv("london_borough_road_centrality.csv", index=False)

In [ ]:
print(df_results)